In [40]:
from brt.router.utils import generate_dst_indices
import torch

dst_num = 4
gates = torch.randn((4, 4)).cuda()
print(gates)
topk_indices = torch.topk(gates, k=1, dim=1).indices
print(topk_indices)
route_indices = (
    torch.zeros(
        gates.size(0),
        dst_num,
        dtype=torch.int64,
        device=gates.device,
    )
    .scatter_(1, topk_indices, 1)
    .cuda()
)
print(route_indices)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
print(supported_capacities)

local_indices, dst_loads = generate_dst_indices(route_indices, supported_capacities)
print(local_indices)
print(dst_loads)

supported_capacities = None

local_indices, dst_loads = generate_dst_indices(route_indices, supported_capacities)
print(local_indices)
print(dst_loads)


tensor([[ 0.1487, -1.0308, -0.6591,  0.1805],
        [-0.6623,  0.0143,  1.1613,  1.2699],
        [ 0.7879, -1.0277,  2.4124,  0.4707],
        [-1.5529, -0.5595, -0.5892, -1.4064]], device='cuda:0')
tensor([[3],
        [3],
        [2],
        [1]], device='cuda:0')
tensor([[0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0]], device='cuda:0')
tensor([ 1,  2,  4,  8, 16], device='cuda:0', dtype=torch.int32)
tensor([[0, 0, 0, 1],
        [0, 0, 0, 2],
        [0, 0, 1, 0],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([1, 1, 1, 2], dtype=torch.int32)
tensor([[0, 0, 0, 1],
        [0, 0, 0, 2],
        [0, 0, 1, 0],
        [0, 1, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([0, 1, 1, 2], dtype=torch.int32)


In [41]:
from brt.router.utils import generate_dst_indices, generate_src_indices
import torch
import numpy as np
dst_num = 4
gates = torch.randn((4, 4)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

route_indices = (
    torch.zeros(
        gates.size(0),
        dst_num,
        dtype=torch.int32,
        device=gates.device,
    )
    .scatter_(1, topk_indices, 1)
    .cuda()
)
print(route_indices)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
supported_capacities = None

local_indices, dst_loads = generate_dst_indices(route_indices, supported_capacities)
print(local_indices)
print(dst_loads)
print(type(dst_loads))


local_indices, dst_loads = generate_src_indices(route_indices, supported_capacities)
print(local_indices)
print(dst_loads)
print(type(dst_loads))


tensor([[1, 0, 0, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([[1, 0, 0, 0],
        [0, 0, 1, 0],
        [2, 0, 0, 0],
        [3, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([3, 0, 1, 0], dtype=torch.int32)
<class 'torch.Tensor'>
tensor([[0, 0, 1, 0],
        [2, 0, 0, 0],
        [3, 0, 0, 0],
        [0, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([3, 0, 1, 0], dtype=torch.int32)
<class 'torch.Tensor'>


In [42]:
from brt._C.router import dispatch_with_dst_indices_1d
import torch

in_data = torch.randn(4, 4, 4, dtype=torch.float32, requires_grad=True).cuda()

total_load = torch.sum(dst_loads).item()
out_data = torch.zeros((total_load, 4), dtype=in_data.dtype).cuda()

new_gates = torch.ones_like(gates, dtype=torch.float32).cuda()

print(in_data)
print(local_indices)
print(dst_loads)
out_data = dispatch_with_dst_indices_1d(
    in_data,
    local_indices,
    dst_loads,
)

print(out_data)


tensor([[[-0.3308,  0.1240,  1.0195, -0.2159],
         [ 0.0225,  1.5018, -0.3679, -0.1782],
         [-0.7061,  1.7608,  1.7714,  0.4425],
         [-2.9409,  0.5268, -1.3022, -0.2594]],

        [[ 0.1721, -0.8454,  0.5393, -1.3184],
         [-0.2518, -0.6651, -0.9887,  0.2620],
         [ 0.2734,  1.8939, -0.9966, -1.6682],
         [-1.9212,  1.5239,  1.1228,  0.2153]],

        [[-0.3862, -0.2001, -0.2069, -1.1311],
         [ 0.1643,  0.1459,  0.8326,  0.1093],
         [ 1.0507, -0.4440,  0.1163, -0.2765],
         [-0.7782,  0.9823, -0.4397,  0.0120]],

        [[-0.2390, -0.7391, -0.1453, -1.3747],
         [ 0.7101, -1.3181,  0.4332,  1.6508],
         [ 0.2804,  0.0571, -1.0369,  0.0058],
         [ 0.1341,  1.1219,  1.2818, -0.2444]]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([[0, 0, 1, 0],
        [2, 0, 0, 0],
        [3, 0, 0, 0],
        [0, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([3, 0, 1, 0], dtype=torch.int32)


RuntimeError: loads.is_cuda() INTERNAL ASSERT FAILED at "/home/yichuanjiaoda/brainstorm_project/brainstorm/src/backend/torch/router.cc":130, please report a bug to PyTorch. loads must be a CUDA tensor

In [ ]:
from brt._C.router import combine_with_src_indices

final_data = combine_with_src_indices(out_data, local_indices, dst_loads)

print(final_data)
print(torch.allclose(final_data, in_data))
print(final_data.shape)

tensor([[[-0.9770, -1.1473, -1.0334, -0.8232],
         [-0.5851, -1.3774,  1.2046, -1.2152],
         [-1.0122, -0.6246,  1.0016,  1.6380],
         [ 1.9609,  0.6950, -1.5311,  1.5943]],

        [[-1.3046,  1.0876,  0.6222,  0.4509],
         [-1.3509,  0.0613, -0.7699, -0.5236],
         [ 0.0943,  0.9361,  0.5199,  0.4449],
         [-0.9209,  0.1338,  0.9276, -1.0284]],

        [[ 0.0651, -0.5160,  1.7939, -1.1250],
         [ 0.7448,  0.4920, -1.0126,  2.8997],
         [ 1.5567,  0.9394, -0.1925,  0.1449],
         [-1.5481,  0.1049,  1.3766, -1.9229]],

        [[ 0.0229,  0.1393, -0.0357,  0.1752],
         [-1.0998,  0.9090, -1.3338, -0.3332],
         [ 0.3678,  0.6949, -0.8650, -0.0833],
         [-0.6429, -0.0560, -0.3000, -0.8104]]], device='cuda:0')
True
torch.Size([4, 4, 4])
